<a href="https://colab.research.google.com/github/Matt-Brigida/FIN_420_Financial_Analytics_Colab/blob/master/week_2_session_3_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can Python Interact with the Workhorse of Banking and Finance?

Banks generally store loan and other information in large relational databases.  The *SQL* Language is often used by financial analysts to retreive data from these databases.

For example, say you offer auto loans.  You can assign a unique identifier (key) to each loan application.  This key serves as a row number and each columns records a detail about the application---loan amount, income, credit score, etc.

If the loan is granted you can record each payment made toward this loan (identified by the unique loan identifier).  If the loan becomes delinquent or defaults, you record the dates of non-payment.

You then have a database loan application with loan performance.  This is clearly useful to analyze which loans tend to be deliquent or default.

Similarly, mortgage applications and payments are stored in large SQL databases.  As an example of a mortgage analysis employing SQL, [see this post](https://toddwschneider.com/posts/mortgages-are-about-math-open-source-loan-level-analysis-of-fannie-and-freddie/
) and [this GitHub repo](https://github.com/toddwschneider/agency-loan-level).

Normally the financial analyst who is going to do this analysis is responsible for extracting the data from the database herself.  Thus, you can consider what we learn here as the first step in many of your analyses, and therefore all the analysis that follows is dependent on getting this right.



We are going to use Pandas to interect with the databse.

In [ ]:
import pandas as pd

## Create Database Connection

We'll use *SQLAlchemy* to create a connection object to a database.  Below we create a *SQLite* database in RAM.  We'll load tables into this database and then execute queries.

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

## Get Spreadsheet Tables

The following is to fix an ssl cert issue:

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
wacc = pd.read_excel("http://www.stern.nyu.edu/~adamodar/pc/datasets/wacc.xls", sheet_name=1, skiprows=18)
wacc

,Industry Name,Number of Firms,Beta,Cost of Equity,E/(D+E),Std Dev in Stock,Cost of Debt,Tax Rate,After-tax Cost of Debt,D/(D+E),Cost of Capital,Cost of Capital (Local Currency)
0,Advertising,58,1.631738,0.135725,0.689708,0.527221,0.0588,0.063905,0.044100,0.310292,0.107295,0.107295
1,Aerospace/Defense,77,1.414182,0.122802,0.793291,0.375555,0.0550,0.086028,0.041250,0.206709,0.105945,0.105945
2,Air Transport,21,1.415965,0.122908,0.349242,0.377275,0.0550,0.104718,0.041250,0.650758,0.069769,0.069769
3,Apparel,39,1.324697,0.117487,0.659751,0.385087,0.0550,0.120381,0.041250,0.340249,0.091547,0.091547
4,Auto & Truck,31,1.540598,0.130312,0.665819,0.526141,0.0588,0.029974,0.044100,0.334181,0.101501,0.101501
...,...,...,...,...,...,...,...,...,...,...,...,...
91,Trucking,35,1.545124,0.130580,0.694861,0.411713,0.0550,0.147859,0.041250,0.305139,0.103322,0.103322
92,Utility (General),15,0.635139,0.076527,0.574135,0.149708,0.0473,0.132030,0.035475,0.425865,0.059045,0.059045
93,Utility (Water),16,1.152415,0.107253,0.697367,0.279613,0.0550,0.084454,0.041250,0.302633,0.087279,0.087279
94,Total Market,7165,1.157270,0.107542,0.651373,0.413733,0.0550,0.075198,0.041250,0.348627,0.084431,0.084431


<!---reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
-->

## Create Database Tables

The following creates a table call `wacc` in our database.

In [ ]:
wacc.to_sql('wacc', con=engine)

## List Tables in the Database

We can see the contents of our databse with:

In [ ]:
engine.table_names()

<ipython-input-7-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['wacc']

## Query the Database

We can now run `SELECT` queries on our one table database.  The following selects all columns (all is denoted by `*`) from the `wacc` table where each row return must have a `Beta` greater than 1.2 and a `Cost of Equity` less than 0.07.

**EDIT** (3/1/2023):  The data has been updated and there are no longer any industries which has a Beta greater than 1.2 and Cost of Equity less than 0.7.  Can you think of why given the present macroeconomic environment?

So below use Beta greater than 1 and Cost of Equity less than 0.12.

In [ ]:
pd.DataFrame(engine.execute("SELECT * FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,4,Auto & Truck,19,1.282826,0.069849,0.721159,0.452370,0.02998,0.078599,0.021885,0.278841,0.056475,0.103314
1,5,Auto Parts,52,1.203498,0.066105,0.804045,0.431649,0.02998,0.073538,0.021885,0.195955,0.057440,0.104322
2,42,Hospitals/Healthcare Facilities,32,1.283102,0.069862,0.501498,0.492114,0.02998,0.081600,0.021885,0.498502,0.045946,0.092318
3,53,Oil/Gas (Integrated),3,1.260614,0.068801,0.693989,0.263884,0.02580,0.085425,0.018834,0.306011,0.053511,0.100218
4,56,Oilfield Svcs/Equip.,135,1.207736,0.066305,0.563579,0.502675,0.02998,0.011881,0.021885,0.436421,0.046919,0.093335
5,62,R.E.I.T.,238,1.205879,0.066217,0.565848,0.324047,0.02580,0.012648,0.018834,0.434152,0.045646,0.092005
6,75,Retail (Special Lines),85,1.282003,0.069811,0.674496,0.490095,0.02998,0.080605,0.021885,0.325504,0.054211,0.100949


Of course if we had a Pandas `DataFrame` of the data, we could return these rows without the need for SQL.  The point here is the data is housed in a SQL database and not in Python.  So we are instructing the SQL database to filter on our requirements and then pass the result to Python/Pandas.

You might think why bother, you can just import the whole table into Pandas and then filter it.  Often the table is very large, so this would be an extremely inefficient approach.

We can select one column with:

In [ ]:
pd.DataFrame(engine.execute("SELECT `Cost of Debt` FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall())

,0
0,0.02998
1,0.02998
2,0.02998
3,0.02580
4,0.02998
5,0.02580
6,0.02998


Or multiple columns:

In [ ]:
pd.DataFrame(engine.execute("SELECT `Tax Rate`,`Cost of Debt` FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall())

,0,1
0,0.078599,0.02998
1,0.073538,0.02998
2,0.081600,0.02998
3,0.085425,0.02580
4,0.011881,0.02998
5,0.012648,0.02580
6,0.080605,0.02998


We can see how many rows have been returned with:

In [ ]:
pd.DataFrame(engine.execute("SELECT COUNT(*) FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall())

,0
0,7


Or calculate a column sum with:

In [ ]:
pd.DataFrame(engine.execute("SELECT SUM(`Cost of Debt`) FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall())

,0
0,0.2015


Other metrics, such as the average, cal be calculated similarly (`avg()`).

## More Tables

Let's add more tables to our database:

In [ ]:
tax_rate_url = "http://www.stern.nyu.edu/~adamodar/pc/datasets/taxrate.xls"  # skip 8 rows, data on sheet 2
debt_url = "http://www.stern.nyu.edu/~adamodar/pc/datasets/dbtfund.xls"  # skip 7 rows, data on sheet 2

In [ ]:
tax = pd.read_excel(tax_rate_url, skiprows=8, sheet_name=1)
debt = pd.read_excel(debt_url, skiprows=7, sheet_name=1)

In [ ]:
tax.columns

In [ ]:
debt.columns

In [ ]:
tax.to_sql('tax', con=engine)
debt.to_sql('debt', con=engine)

In [ ]:
engine.table_names()

## Merging Tables

The following query merges all tables.

In [ ]:
join_query = "SELECT * FROM wacc LEFT OUTER JOIN tax ON tax.`Industry Name`=wacc.`Industry Name` LEFT OUTER JOIN debt ON debt.`Industry Name`=tax.`Industry Name`"

In [ ]:
pd.DataFrame(engine.execute(join_query).fetchall())

# Other Databases

Above we have created and queried a `SQLite` database, however Python can connect to many other popular relational databases such as `PostgreSQL`, etc.  

Python can also interact with `NoSQL` and `key-value` type databases such as `MongoDB` and `Redis` among others.

# Exercise

Select only the `Tax Rate` column from the wacc table however keep the contraints where each row return must have a `Beta` greater than 1.0 and a `Cost of Equity` less than 0.12.  Have your SELECT query calculate and return the average `Tax Rate`.

<!---## Answer-->


<!---The average Tax Rate is 6.0614%-->

<!---pd.DataFrame(engine.execute("SELECT `Tax Rate` FROM wacc WHERE Beta > 1.2 AND `Cost of Equity` < 0.07").fetchall()).mean()-->